---
---

### Kodun yapısını parça parça inceleyip:

* Neyi, neden böyle tanımlıyoruz?

* Hangi parametre neyi kontrol ediyor?

* Hangi durumda residual devreye giriyor?

* Modele nasıl entegre edilir?

bunların hepsini netleştirelim.

---
---

## Kod:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MBConv(nn.Module):
    """
    Inverted Bottleneck (MBConv) bloğu.

    Yapı:
    - 1x1 expansion conv (C_in -> C_exp)
    - 3x3 depthwise conv (C_exp -> C_exp, groups=C_exp)
    - 1x1 projection conv (C_exp -> C_out)
    - opsiyonel residual (stride=1 ve C_in==C_out ise)
    """
    def __init__(self, c_in, c_out, expansion_factor=6, stride=1, kernel_size=3):
        super().__init__()
        self.stride = stride
        self.use_residual = (stride == 1 and c_in == c_out)
        
        c_exp = c_in * expansion_factor
        
        # 1) Expansion 1x1 conv
        self.expand = nn.Sequential(
            nn.Conv2d(c_in, c_exp, kernel_size=1, bias=False),
            nn.BatchNorm2d(c_exp),
            nn.SiLU(inplace=True)  # Swish benzeri; ReLU6 da kullanılabilir
        )
        
        # 2) Depthwise 3x3 conv
        padding = kernel_size // 2
        self.depthwise = nn.Sequential(
            nn.Conv2d(
                c_exp,
                c_exp,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                groups=c_exp,   # depthwise
                bias=False
            ),
            nn.BatchNorm2d(c_exp),
            nn.SiLU(inplace=True)
        )
        
        # 3) Projection 1x1 conv (linear)
        self.project = nn.Sequential(
            nn.Conv2d(c_exp, c_out, kernel_size=1, bias=False),
            nn.BatchNorm2d(c_out)
            # Aktivasyon YOK (linear bottleneck)
        )
    
    def forward(self, x):
        identity = x
        
        out = self.expand(x)
        out = self.depthwise(out)
        out = self.project(out)
        
        if self.use_residual:
            out = out + identity
        
        return out

### 1. Constructor seviyesinde mantık
Parametreler:

* c_in → giriş kanal sayısı (in_channels)

* c_out → çıkış kanal sayısı (out_channels)

* expansion_factor → genişletme katsayısı (t). MobilenetV2’de tipik olarak 4 veya 6

* stride → bu bloğun downsample yapıp yapmayacağını belirler (1 veya 2)

* kernel_size → genelde 3, bazı çalışmalarda 5 de kullanılıyor


**İlk satırlar:**
```python
self.stride = stride
self.use_residual = (stride == 1 and c_in == c_out)
```


Residual (skip connection) sadece şu durumda mantıklı:

* Uzamsal boyut değişmiyorsa → stride == 1

* Kanal sayısı değişmiyorsa → c_in == c_out

- Aksi halde x ile out’un shape’i farklı olur, direkt toplayamazsın.


**Yani bu satır:**
```python
self.use_residual = (stride == 1 and c_in == c_out)
```


demek ki:

* Bloğu MBConv(16, 16, stride=1) gibi tanımlarsan → residual aktif

* MBConv(16, 24, stride=2) gibi downsample/kanal değişimi varsa → residual devre dışı

### Expansion kanalı:
```python
c_exp = c_in * expansion_factor
```


Girişteki bilgi önce yüksek boyuta taşınıyor.

- c_in = 16, expansion_factor = 6 ise → c_exp = 96.

- Amaç: Depthwise conv’u geniş bir temsil uzayında yapmak (daha fazla ifade gücü).

## 2. Expansion bloğu (1×1 conv)
```python
self.expand = nn.Sequential(
    nn.Conv2d(c_in, c_exp, kernel_size=1, bias=False),
    nn.BatchNorm2d(c_exp),
    nn.SiLU(inplace=True)
)
```


* 1×1 Conv: C_in → C_exp

* BN: Eğitim stabilitesi

* Aktivasyon: SiLU (Swish). Yerine ReLU6 de kullanılabilir.

Bu aşama:

* ResNet’teki bottleneck’in sonundaki 1×1 conv’a karşılık gelir.

* Burada ise başta kullanıldığı için “inverted” diyoruz.

Girdi/Çıktı:

**Girdi: (N, c_in, H, W)**

**Çıktı: (N, c_exp, H, W)**

## 3. Depthwise bloğu (3×3, groups=c_exp)
```python
padding = kernel_size // 2
self.depthwise = nn.Sequential(
    nn.Conv2d(
        c_exp,
        c_exp,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        groups=c_exp,
        bias=False
    ),
    nn.BatchNorm2d(c_exp),
    nn.SiLU(inplace=True)
)
```

### Buradaki kritik noktalar:

* in_channels = c_exp

* out_channels = c_exp

* groups = c_exp → Depthwise convolution (her kanal kendi filtresiyle)

Bu katman:

* Uzamsal işlemi yapıyor (3×3 veya 5×5)

* stride = 1 ise boyut korunur

* stride = 2 ise high-level: downsample (H,W → H/2,W/2)

Depthwise’in burada kullanılma nedeni:

* Geniş temsil (c_exp) üzerinde uzamsal işlem yaparken,

* standart conv’a göre çok daha ucuz FLOPs kullanmak.

## 4. Projection bloğu (1×1, linear)
```python
self.project = nn.Sequential(
    nn.Conv2d(c_exp, c_out, kernel_size=1, bias=False),
    nn.BatchNorm2d(c_out)
    # Aktivasyon YOK
)
```


* c_exp → c_out dönüşümü

* Kanal sayısını tekrar “daraltıyoruz”.

* Aktivasyon kullanılmaması, linear bottleneck fikri.

Buradaki amaç:

* c_exp uzayında non-lineerlikleri uyguladın (SiLU/ReLU6’lar ile)

* Son projeksiyon, bilgiyi lineer bir uzaya geri taşıyor

* Böylece ReLU gibi aktivasyonların getirdiği bilgi kaybı (clipping) proje katmanında yaşanmıyor.

Girdi/Çıktı:

**Girdi: (N, c_exp, H', W')**

**Çıktı: (N, c_out, H', W')**

## 5. forward fonksiyonu ve residual mantığı
```python
def forward(self, x):
    identity = x
    
    out = self.expand(x)
    out = self.depthwise(out)

    out = self.project(out)
    
    if self.use_residual:
        out = out + identity
    
    return out
```


Sıra:

* expand → kanalı genişlet

* depthwise → uzamsal işlem

* project → kanalı daralt

Sonra koşullu residual:

* Eğer use_residual == True ise:

* out ve identity aynı shape’te → toplanır

* Bloğun çıktısı: x + F(x) olur

Bu yapı:

* Derin ağlarda gradient akışını rahatlatır

* MobileNetV2, EfficientNet vb. mimarilerin “residual MBConv” blokları tam olarak budur.

## 6. Bu bloğu bir modelde nasıl kullanırsın?

Basit bir örnek model:
```python
class SmallMBConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # İlk basit conv ile giriş özelliğini hazırlayalım
        self.stem = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.SiLU(inplace=True)
        )
        
        # Ardından MBConv blokları
        self.block1 = MBConv(c_in=16, c_out=16, expansion_factor=6, stride=1)
        self.block2 = MBConv(c_in=16, c_out=24, expansion_factor=6, stride=2)  # downsample
        self.block3 = MBConv(c_in=24, c_out=24, expansion_factor=6, stride=1)
        
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc   = nn.Linear(24, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Küçük test
x = torch.randn(4, 3, 32, 32)
model = SmallMBConvNet(num_classes=10)
y = model(x)
print("Çıktı şekli:", y.shape)
```

Burada MobileNetV2 tarzı bir iskeleti, basitleştirilmiş hâliyle görmüş oluyorsun:

* Stem conv

* Birkaç MBConv bloğu

* Global Average Pool + FC

## 7. Tasarım kararlarını anlaman için kısa özet

* expansion_factor → genişletme oranı; ne kadar büyükse temsil gücü artar, maliyet de artar.

* depthwise kısmı → 3×3 uzamsal filtrelemeyi ucuz yapar (groups=c_exp).

* project kısmı → lineer olarak daraltır, klasik “bottleneck” çıkışı.

* use_residual → sadece stride=1 ve c_in==c_out iken residual eklenir.

Aktivasyonlar:

* expand ve depthwise sonrasında var

* project sonunda yok (linear bottleneck)

---
---

# 📌 MBConv (Inverted Bottleneck) — Boyutlandırma Mantığı

MBConv (MobileNetV2 / EfficientNet bloğu) içinde **kanal boyutları (C)** ve **uzamsal boyutlar (H×W)** belirli kurallara göre değişir.  
Aşağıdaki özet, tek başına tüm MBConv boyutlandırmasını anlaman için yeterlidir.

---

## 1. Kanal Boyutları (C_in → C_exp → C_out)

Bir MBConv bloğu kanal yönünden şu sırayı izler:

### **1) Giriş**
- Şekil: **(N, C_in, H, W)**

### **2) Expansion (1×1 Conv)**
- `C_exp = C_in × expansion_factor`
- Yalnızca kanal sayısı değişir.
- Çıktı: **(N, C_exp, H, W)**

### **3) Depthwise (3×3 veya 5×5, groups=C_exp)**
- Kanal sayısı değişmez.
- Çıktı: **(N, C_exp, H', W')**

### **4) Projection (1×1 Conv, Linear)**
- Kanal daraltma yapılır.
- Çıktı: **(N, C_out, H', W')**

### **5) Residual Bağlantı**
Aşağıdaki iki koşul sağlanıyorsa residual eklenir:
- `stride == 1`
- `C_in == C_out`

Bu durumda:  
**(N, C_out, H, W) = input + projected_output**

---

## 2. Uzamsal Boyutlar (H×W)

MBConv içinde uzamsal boyut sadece **depthwise** katmanda değişir.

Depthwise için genel konvolüsyon formülü:

\[
H_{\text{out}} =
\left\lfloor \frac{H_{\text{in}} + 2\cdot \text{padding} - \text{kernel}}{\text{stride}} \right\rfloor + 1
\]

MBConv’de:

- `kernel_size = 3` veya `5`
- `padding = kernel_size // 2`

Bu nedenle:

---

### **Stride = 1 → Boyut sabit**

\[
H_{\text{out}} = H,\quad W_{\text{out}} = W
\]

Örnek:  
`(32×32) → (32×32)`

---

### **Stride = 2 → Boyut yarıya iner**

\[
H_{\text{out}} \approx \frac{H}{2},\quad W_{\text{out}} \approx \frac{W}{2}
\]

Örnek:  
`(32×32) → (16×16)`

---

## 3. Örnek Akışlar

### **Örnek 1 — Stride 1, Residual Var**
MBConv(16 → 16, expansion_factor=6, stride=1)

1. Girdi: **(N, 16, 32, 32)**
2. Expand: **(N, 96, 32, 32)**
3. Depthwise: **(N, 96, 32, 32)**
4. Project: **(N, 16, 32, 32)**
5. Residual: evet  
   Çıkış: **(N, 16, 32, 32)**

---

### **Örnek 2 — Stride 2, Residual Yok**
MBConv(16 → 24, expansion_factor=6, stride=2)

1. Girdi: **(N, 16, 32, 32)**
2. Expand: **(N, 96, 32, 32)**
3. Depthwise(stride=2): **(N, 96, 16, 16)**
4. Project: **(N, 24, 16, 16)**
5. Residual: hayır (`stride=2` olduğu için)

Çıkış: **(N, 24, 16, 16)**

---

## 4. Özet — MBConv Boyutlandırma Kuralları

- **Kanal genişletme:**  
  `C_exp = C_in × expansion_factor`

- **Uzamsal boyut:**  
  - Stride 1 → aynı  
  - Stride 2 → yarıya iner  

- **Residual bağlanır:**  
  - `stride == 1` ve `C_in == C_out`

- **Projection lineer tutulur:**  
  Aktivasyon yoktur → bilgi kaybını azaltır.

Bu özetle artık her MBConv bloğunun çıktı boyutunu zihninde otomatik hesaplayabilirsin.
